# 1장: 기본 프롬프트 구조

- [수업](#lesson)
- [연습 문제](#exercises)
- [예제 플레이그라운드](#example-playground)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 도우미 함수를 설정하세요.

In [ ]:
!pip install anthropic

# Python의 내장 정규 표현식 라이브러리 가져오기
import re
import anthropic

# IPython 저장소에서 API_KEY 및 MODEL_NAME 변수 검색
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---

## 수업

Anthropic은 레거시 [Text Completions API](https://docs.anthropic.com/claude/reference/complete_post)와 현재의 [Messages API](https://docs.anthropic.com/claude/reference/messages_post) 두 가지 API를 제공합니다. 이 튜토리얼에서는 Messages API만 사용할 것입니다.

Messages API를 사용하여 Claude를 호출할 때 최소한 다음 매개변수가 필요합니다:

- `model`: 호출하려는 모델의 [API 모델 이름](https://docs.anthropic.com/claude/docs/models-overview#model-recommendations)

- `max_tokens`: 생성을 중지하기 전에 생성할 최대 토큰 수. Claude는 이 최대값에 도달하기 전에 중지할 수 있습니다. 이 매개변수는 생성할 절대 최대 토큰 수만 지정합니다. 또한, 이것은 *강제* 중지이므로 Claude가 단어 중간이나 문장 중간에 생성을 중지하게 할 수 있습니다.

- `messages`: 입력 메시지 배열. 우리 모델은 번갈아 가며 `user`와 `assistant` 대화 차례에서 작동하도록 훈련되었습니다. 새 `Message`를 만들 때 messages 매개변수로 이전 대화 차례를 지정하면 모델이 대화의 다음 `Message`를 생성합니다.
  - 각 입력 메시지는 `role`과 `content`가 있는 객체여야 합니다. 단일 `user` 역할 메시지를 지정하거나 여러 `user`와 `assistant` 메시지를 포함할 수 있습니다(이 경우 번갈아 가야 함). 첫 번째 메시지는 항상 user `role`을 사용해야 합니다.

다음과 같은 선택적 매개변수도 있습니다:

- `system`: 시스템 프롬프트 - 이에 대해서는 아래에서 더 자세히 설명합니다.
  
- `temperature`: Claude 응답의 다양성 정도. 이 수업과 연습 문제에서는 `temperature`를 0으로 설정했습니다.

모든 API 매개변수의 전체 목록은 [API 문서](https://docs.anthropic.com/claude/reference/messages_post)를 참조하세요.

### 예제

Claude가 올바르게 형식이 지정된 프롬프트에 어떻게 응답하는지 살펴보겠습니다. 다음 각 셀에 대해 셀을 실행(`shift+enter`)하면 Claude의 응답이 블록 아래에 나타납니다.

In [ ]:
# 프롬프트
PROMPT = "안녕 Claude, 어떻게 지내?"

# Claude의 응답 출력
print(get_completion(PROMPT))

In [ ]:
# 프롬프트
PROMPT = "바다의 색깔을 알려줄 수 있어?"

# Claude의 응답 출력
print(get_completion(PROMPT))

In [ ]:
# 프롬프트
PROMPT = "셀린 디온은 몇 년에 태어났어?"

# Claude의 응답 출력
print(get_completion(PROMPT))

이제 올바른 Messages API 형식을 포함하지 않는 프롬프트를 살펴보겠습니다. 이러한 잘못된 형식의 프롬프트에 대해 Messages API는 오류를 반환합니다.

먼저, `messages` 배열에 `role`과 `content` 필드가 없는 Messages API 호출의 예를 살펴보겠습니다.

In [ ]:
# Claude의 응답 가져오기
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Claude의 응답 출력
print(response[0].text)

다음은 `user`와 `assistant` 역할을 번갈아 가지 않는 프롬프트입니다.

In [ ]:
# Claude의 응답 가져오기
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "셀린 디온은 몇 년에 태어났어?"},
          {"role": "user", "content": "그리고, 그녀에 대한 다른 사실도 알려줄 수 있어?"}
        ]
    )

# Claude의 응답 출력
print(response[0].text)

`user`와 `assistant` 메시지는 **반드시 번갈아** 가야 하며, 메시지는 **반드시 `user` 차례로 시작**해야 합니다. 프롬프트에 여러 `user` 및 `assistant` 쌍을 포함할 수 있습니다(마치 다중 턴 대화를 시뮬레이션하는 것처럼). 또한 Claude가 중단한 지점부터 계속할 수 있도록 마지막 `assistant` 메시지에 단어를 넣을 수도 있습니다(이에 대해서는 이후 장에서 더 자세히 다룹니다).

#### 시스템 프롬프트

**시스템 프롬프트**를 사용할 수도 있습니다. 시스템 프롬프트는 "사용자" 차례에서 질문이나 작업을 제시하기 전에 **Claude에게 컨텍스트, 지침 및 가이드라인을 제공**하는 방법입니다.

구조적으로, 시스템 프롬프트는 `user` 및 `assistant` 메시지 목록과 별도로 존재하므로 별도의 `system` 매개변수에 속합니다(노트북의 [설정](#setup) 섹션에서 `get_completion` 도우미 함수의 구조를 살펴보세요).

이 튜토리얼에서 시스템 프롬프트를 사용할 수 있는 곳에서는 완료 함수에 `system` 필드를 제공했습니다. 시스템 프롬프트를 사용하지 않으려면 `SYSTEM_PROMPT` 변수를 빈 문자열로 설정하면 됩니다.

#### 시스템 프롬프트 예제

In [ ]:
# 시스템 프롬프트
SYSTEM_PROMPT = "당신의 답변은 항상 대화를 더 발전시키는 일련의 비판적 사고 질문이어야 합니다(질문에 대한 답변을 제공하지 마세요). 실제로 사용자 질문에 답하지 마세요."

# 프롬프트
PROMPT = "하늘이 왜 파란색인가요?"

# Claude의 응답 출력
print(get_completion(PROMPT, SYSTEM_PROMPT))

왜 시스템 프롬프트를 사용할까요? **잘 작성된 시스템 프롬프트는 Claude의 성능을 향상**시킬 수 있습니다. 예를 들어, Claude가 규칙과 지침을 따르는 능력을 향상시킬 수 있습니다. 자세한 내용은 Claude와 함께 [시스템 프롬프트를 사용하는 방법](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts)에 대한 문서를 참조하세요.

이제 몇 가지 연습 문제를 살펴보겠습니다. 위의 내용을 변경하지 않고 수업 프롬프트를 실험해보고 싶다면, 수업 노트북의 맨 아래로 스크롤하여 [**예제 플레이그라운드**](#example-playground)를 방문하세요.

---

## 연습 문제

- [연습 문제 1.1 - 셋까지 세기](#exercise-11---counting-to-three)
- [연습 문제 1.2 - 시스템 프롬프트](#exercise-12---system-prompt)

### 연습 문제 1.1 - 셋까지 세기

올바른 `user` / `assistant` 형식을 사용하여 아래의 `PROMPT`를 편집하여 Claude가 **셋까지 세도록** 만드세요. 출력은 또한 당신의 해결책이 올바른지 여부를 나타냅니다.

In [ ]:
# 프롬프트 - 변경해야 할 유일한 필드입니다
PROMPT = "[이 텍스트를 바꾸세요]"

# Claude의 응답 가져오기
response = get_completion(PROMPT)

# 연습 문제 정확성을 평가하는 함수
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Claude의 응답과 해당 등급 출력
print(response)
print("\n--------------------------- 채점 ---------------------------")
print("이 연습 문제가 올바르게 해결되었습니다:", grade_exercise(response))

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

### 연습 문제 1.2 - 시스템 프롬프트

Claude가 3살 아이처럼 응답하도록 `SYSTEM_PROMPT`를 수정하세요.

In [ ]:
# 시스템 프롬프트 - 변경해야 할 유일한 필드입니다
SYSTEM_PROMPT = "[이 텍스트를 바꾸세요]"

# 프롬프트
PROMPT = "하늘은 얼마나 큰가요?"

# Claude의 응답 가져오기
response = get_completion(PROMPT, SYSTEM_PROMPT)

# 연습 문제 정확성을 평가하는 함수
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Claude의 응답과 해당 등급 출력
print(response)
print("\n--------------------------- 채점 ---------------------------")
print("이 연습 문제가 올바르게 해결되었습니다:", grade_exercise(response))

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

### 축하합니다!

지금까지 모든 연습 문제를 해결했다면, 다음 장으로 넘어갈 준비가 되었습니다. 즐거운 프롬프팅 되세요!

---

## 예제 플레이그라운드

이 영역에서는 이 수업에서 보여준 프롬프트 예제를 자유롭게 실험하고 프롬프트를 조정하여 Claude의 응답에 어떤 영향을 미치는지 확인할 수 있습니다.

In [ ]:
# 프롬프트
PROMPT = "안녕 Claude, 어떻게 지내?"

# Claude의 응답 출력
print(get_completion(PROMPT))

In [ ]:
# 프롬프트
PROMPT = "바다의 색깔을 알려줄 수 있어?"

# Claude의 응답 출력
print(get_completion(PROMPT))

In [ ]:
# 프롬프트
PROMPT = "셀린 디온은 몇 년에 태어났어?"

# Claude의 응답 출력
print(get_completion(PROMPT))

In [ ]:
# Claude의 응답 가져오기
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Claude의 응답 출력
print(response[0].text)

In [ ]:
# Claude의 응답 가져오기
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "셀린 디온은 몇 년에 태어났어?"},
          {"role": "user", "content": "그리고, 그녀에 대한 다른 사실도 알려줄 수 있어?"}
        ]
    )

# Claude의 응답 출력
print(response[0].text)

In [ ]:
# 시스템 프롬프트
SYSTEM_PROMPT = "당신의 답변은 항상 대화를 더 발전시키는 일련의 비판적 사고 질문이어야 합니다(질문에 대한 답변을 제공하지 마세요). 실제로 사용자 질문에 답하지 마세요."

# 프롬프트
PROMPT = "하늘이 왜 파란색인가요?"

# Claude의 응답 출력
print(get_completion(PROMPT, SYSTEM_PROMPT))